### Queries to develop the star-based schema

Create database schema and get the columns in the correct data types.

The notebook is to be run top-down.

In [ ]:
import psycopg2
from sqlalchemy import create_engine, inspect, text
from database_utils import DatabaseConnector

connector = DatabaseConnector('db_creds.yaml')


### Task One


In [ ]:
alter_command = """
ALTER TABLE orders_table
    ALTER COLUMN date_uuid TYPE UUID USING (date_uuid::UUID),
    ALTER COLUMN user_uuid TYPE UUID USING (user_uuid::UUID),
    ALTER COLUMN card_number TYPE VARCHAR(255),
    ALTER COLUMN store_code TYPE VARCHAR(255),
    ALTER COLUMN product_code TYPE VARCHAR(255),
    ALTER COLUMN product_quantity TYPE SMALLINT USING (product_quantity::SMALLINT);
"""

connector.local_connection.execute(text(alter_command))

print("Column data types changed successfully.")

Column data types changed successfully.


### Task Two

In [ ]:
alter_command = """
UPDATE dim_users
SET user_uuid = NULL
WHERE user_uuid::text !~ '-';

DELETE FROM dim_users WHERE user_uuid IS NULL;

ALTER TABLE dim_users
    ALTER COLUMN first_name TYPE VARCHAR(255),
    ALTER COLUMN last_name TYPE VARCHAR(255),
    ALTER COLUMN date_of_birth TYPE DATE USING (date_of_birth::DATE),
    ALTER COLUMN country_code TYPE VARCHAR(16), 
    ALTER COLUMN user_uuid TYPE UUID USING (user_uuid::UUID),
    ALTER COLUMN join_date TYPE DATE USING (join_date::DATE);


"""

connector.local_connection.execute(text(alter_command))

print("Column data types changed successfully.")

Column data types changed successfully.


### Task Three

In [ ]:
alter_command = """ 
UPDATE dim_store_details SET latitude = COALESCE(latitude, lat);
ALTER TABLE dim_store_details DROP COLUMN lat;
"""

connector.local_connection.execute(text(alter_command))
connector.local_connection.commit()
print("Latitudes successfuly merged.")

Latitudes successfuly merged.


In [ ]:
alter_command = """ 
UPDATE dim_store_details
SET staff_numbers = NULL
WHERE staff_numbers ~ '[^0-9]';

UPDATE dim_store_details
SET longitude = NULL
WHERE longitude !~ '^[-]?[0-9]+(\.[0-9]+)?$';

UPDATE dim_store_details
SET latitude = NULL
WHERE latitude !~ '^[-]?[0-9]+(\.[0-9]+)?$';

UPDATE dim_store_details
SET opening_date = NULL
WHERE opening_date !~ '^[0-9]{4}-[0-9]{2}-[0-9]{2}$';


"""

connector.local_connection.execute(text(alter_command))
connector.local_connection.commit()
print("Alphanumeric successfully removed")

Alphanumeric successfully removed


In [ ]:
alter_command = """ 
ALTER TABLE dim_store_details
    ALTER COLUMN longitude TYPE FLOAT USING longitude::FLOAT,
    ALTER COLUMN locality TYPE VARCHAR(64),
    ALTER COLUMN store_code TYPE VARCHAR(255), 
    ALTER COLUMN staff_numbers TYPE SMALLINT USING staff_numbers::SMALLINT,
    ALTER COLUMN opening_date TYPE DATE USING opening_date::DATE,
    ALTER COLUMN store_type TYPE VARCHAR(255),
    ALTER COLUMN latitude TYPE FLOAT USING latitude::FLOAT,
    ALTER COLUMN country_code TYPE VARCHAR(255),
    ALTER COLUMN continent TYPE VARCHAR(255);

UPDATE dim_store_details
SET locality = 'N/A'
WHERE locality IS NULL;
"""

connector.local_connection.execute(text(alter_command))
connector.local_connection.commit()
print("Type successfully updated.")

Type successfully updated.


### TASK FOUR

In [ ]:
alter_command = """
UPDATE dim_products
SET product_price = REPLACE(product_price, '£', '');

"""
connector.local_connection.execute(text(alter_command))
connector.local_connection.commit()
print("£ successfully removed")

£ successfully removed


In [ ]:

alter_command = """
ALTER TABLE dim_products
ADD COLUMN weight_class VARCHAR(20);

UPDATE dim_products
SET weight_class = CASE
    WHEN weight_kg < 2 THEN 'Light'
    WHEN weight_kg >= 2 AND weight_kg < 40 THEN 'Mid_Sized'
    WHEN weight_kg >= 40 AND weight_kg < 140 THEN 'Heavy'
    WHEN weight_kg >= 140 THEN 'Truck_Required'
    ELSE 'Unknown'  -- Optional, to handle any cases not covered above
END;

"""
connector.local_connection.execute(text(alter_command))
connector.local_connection.commit()
print("categories successfully added")

categories successfully added


### TASK FIVE

In [ ]:

alter_command = """

ALTER TABLE dim_products
RENAME COLUMN removed TO still_available;

UPDATE dim_products
SET still_available = CASE
    WHEN still_available = 'Still_avaliable' THEN 'true'
    ELSE 'false'
END;


"""
connector.local_connection.execute(text(alter_command))
connector.local_connection.commit()
print("Categories successfully added")

Categories successfully added


In [ ]:
alter_command = """
UPDATE dim_products
SET product_price = NULL
WHERE product_price !~ '^[-]?[0-9]+(\.[0-9]+)?$';


UPDATE dim_products
SET date_added = NULL
WHERE date_added !~ '^[0-9]{4}-[0-9]{2}-[0-9]{2}$';

UPDATE dim_products
SET uuid = NULL
WHERE uuid !~ '-';

"""
connector.local_connection.execute(text(alter_command))
connector.local_connection.commit()
print("Categories successfully added")

Categories successfully added


In [ ]:
# ALTER COLUMN weight_kg TYPE FLOAT USING weight_kg::FLOAT,
alter_command = """

ALTER TABLE dim_products
    ALTER COLUMN product_price TYPE FLOAT USING product_price::FLOAT,
    ALTER COLUMN \"EAN\" TYPE VARCHAR(20),  
    ALTER COLUMN product_code TYPE VARCHAR(20),  
    ALTER COLUMN date_added TYPE DATE USING date_added::DATE,
    ALTER COLUMN uuid TYPE UUID USING uuid::UUID,
    ALTER COLUMN still_available TYPE BOOL USING still_available::BOOL,
    ALTER COLUMN weight_class TYPE VARCHAR(20);  

"""
connector.local_connection.execute(text(alter_command))
connector.local_connection.commit()
print("Categories successfully added") 

Categories successfully added


### TASK SIX

In [ ]:
alter_command = """
UPDATE dim_date_times
SET month = NULL
WHERE month !~ '^[-]?[0-9]+(\.[0-9]+)?$';

UPDATE dim_date_times
SET year = NULL
WHERE year !~ '^[-]?[0-9]+(\.[0-9]+)?$';

UPDATE dim_date_times
SET day = NULL
WHERE day !~ '^[-]?[0-9]+(\.[0-9]+)?$';

UPDATE dim_date_times
SET timestamp = NULL
WHERE timestamp !~ ':';

UPDATE dim_date_times
SET date_uuid = NULL
WHERE date_uuid !~ '-';

UPDATE dim_date_times
SET time_period = NULL
WHERE time_period !~ '^[A-Za-z]+$';

"""
connector.local_connection.execute(text(alter_command))
connector.local_connection.commit()
print("removed unwanted items")

removed unwanted items


In [ ]:
# ALTER COLUMN weight_kg TYPE FLOAT USING weight_kg::FLOAT,
alter_command = """

ALTER TABLE dim_date_times
    ALTER COLUMN month TYPE VARCHAR(20),  
    ALTER COLUMN year TYPE VARCHAR(10),   
    ALTER COLUMN day TYPE VARCHAR(10),    
    ALTER COLUMN time_period TYPE VARCHAR(20),  
    ALTER COLUMN date_uuid TYPE UUID USING date_uuid::uuid;


"""
connector.local_connection.execute(text(alter_command))
connector.local_connection.commit()
print("Casting done successfully ") 

Casting done successfully 


### TASK SEVEN

In [ ]:
alter_command = """
UPDATE dim_card_details
SET card_number = NULL
WHERE card_number !~ '^[-]?[0-9]+(\.[0-9]+)?$';



UPDATE dim_card_details
SET expiry_date = NULL
WHERE expiry_date !~ '/';

"""
connector.local_connection.execute(text(alter_command))
connector.local_connection.commit()
print("removed unwanted items")

removed unwanted items


In [ ]:
# ALTER COLUMN weight_kg TYPE FLOAT USING weight_kg::FLOAT,
alter_command = """

ALTER TABLE dim_card_details
    ALTER COLUMN card_number TYPE VARCHAR(32),
    ALTER COLUMN expiry_date TYPE VARCHAR(8),
    ALTER COLUMN date_payment_confirmed TYPE DATE;



"""
connector.local_connection.execute(text(alter_command))
connector.local_connection.commit()
print("Casting done successfully ") 

Casting done successfully 


### TASK EIGHT

In [ ]:
alter_command = """
DELETE FROM dim_card_details WHERE card_number IS NULL;


ALTER TABLE dim_card_details
ADD PRIMARY KEY (card_number);

"""
connector.local_connection.execute(text(alter_command))
connector.local_connection.commit()
print("Primary key set successfully ") 


Primary key set successfully 


In [ ]:
alter_command = """
DELETE FROM dim_date_times WHERE date_uuid IS NULL;


ALTER TABLE dim_date_times
ADD PRIMARY KEY (date_uuid);

"""
connector.local_connection.execute(text(alter_command))
connector.local_connection.commit()
print("Primary key set successfully ") 


Primary key set successfully 


In [ ]:
alter_command = """
DELETE FROM dim_users WHERE user_uuid IS NULL;


ALTER TABLE dim_users
ADD PRIMARY KEY (user_uuid);

"""
connector.local_connection.execute(text(alter_command))
connector.local_connection.commit()
print("Primary key set successfully ") 


Primary key set successfully 


In [ ]:
alter_command = """
DELETE FROM dim_store_details WHERE store_code IS NULL;
DELETE FROM dim_store_details WHERE store_code = 'NULL';

ALTER TABLE dim_store_details
ADD PRIMARY KEY (store_code);

"""
connector.local_connection.execute(text(alter_command))
connector.local_connection.commit()
print("Primary key set successfully ") 


Primary key set successfully 


In [ ]:
alter_command = """
DELETE FROM dim_products WHERE product_code IS NULL;
DELETE FROM dim_products WHERE product_code = 'NULL';

ALTER TABLE dim_products
ADD PRIMARY KEY (product_code);

"""
connector.local_connection.execute(text(alter_command))
connector.local_connection.commit()
print("Primary key set successfully ") 


Primary key set successfully 


### TASK NINE

In [ ]:

alter_command = """
ALTER TABLE orders_table 
    ALTER COLUMN date_uuid TYPE UUID USING date_uuid::uuid,
    ALTER COLUMN card_number TYPE VARCHAR(32);

ALTER TABLE orders_table
ADD CONSTRAINT fk_orders_users
FOREIGN KEY (user_uuid) REFERENCES dim_users(user_uuid),
ADD CONSTRAINT fk_orders_products
FOREIGN KEY (product_code) REFERENCES dim_products(product_code),
ADD CONSTRAINT fk_orders_stores
FOREIGN KEY (store_code) REFERENCES dim_store_details(store_code),
ADD CONSTRAINT fk_orders_dates
FOREIGN KEY (date_uuid) REFERENCES dim_date_times(date_uuid),
ADD CONSTRAINT fk_orders_cards
FOREIGN KEY (card_number) REFERENCES dim_card_details(card_number);


"""
connector.local_connection.execute(text(alter_command))
connector.local_connection.commit()
print("Primary key set successfully ") 


Primary key set successfully 
